In [ ]:
import tensorflow as tf

In [ ]:
train_data_path = "file:///C:/SourceCode/GitHub/wayneg/recs-models/data/sampledata/modelSamples.csv"
sample_file_path = tf.keras.utils.get_file("modelSamples.csv", train_data_path)

In [ ]:
def make_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True
    )

    return dataset

In [ ]:
raw_samples_data = make_dataset(sample_file_path)
print(raw_samples_data)

In [ ]:
test_dataset = raw_samples_data.take(1000)
train_dataset = raw_samples_data.skip(1000)

In [ ]:
movie_columns = []
# movie id embedding
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
movie_columns.append(movie_emb_col)

In [ ]:
user_columns = []
# user id embedding
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
user_columns.append(user_emb_col)


In [ ]:

inputs = {
    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32')
}

In [ ]:
# model architecture
movie_deep = tf.keras.layers.DenseFeatures(movie_columns)(inputs)
movie_deep = tf.keras.layers.Dense(units=20, activation='relu')(movie_deep)
movie_deep = tf.keras.layers.Dense(units=20, activation='relu')(movie_deep)

user_deep = tf.keras.layers.DenseFeatures(user_columns)(inputs)
user_deep = tf.keras.layers.Dense(units=20, activation='relu')(user_deep)
user_deep = tf.keras.layers.Dense(units=20, activation='relu')(user_deep)

output_layer = tf.keras.layers.Dot(axes=1)([movie_deep, user_deep])

model = tf.keras.Model(inputs, output_layer)

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
model.fit(train_dataset, epochs=5)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

In [ ]:
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))